## Get all the articles to summarize

In [1]:
import requests

host = "http://144.24.201.133:5000"
articles_json = requests.get(f"{host}/allPapers").json()
print(len(articles_json))

1000


## Instanciate BART

In [2]:
from transformers import BartForConditionalGeneration, AutoTokenizer

model_ckpt = "sshleifer/distilbart-cnn-6-6"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BartForConditionalGeneration.from_pretrained(model_ckpt).cuda()

## Check Cuda comparibility

In [3]:
import torch
from tqdm import tqdm
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Tokenize and filter by number of pages
If a paper has more than 10 pages, it is filtered out as a single summary will be too long for chat gpt input


In [4]:
page_size = 512

paper_rows = []
skipped_papers = []
for paper in tqdm(articles_json):
    tokens = tokenizer(paper['body'], padding='max_length', return_tensors='pt').to(device)
    n_pages = math.ceil(len(tokens["input_ids"][0])/page_size)
    paper_rows.append(
            {
            "link": paper["link"],
            "body":  paper["body"],
            "summary":  "",
            "input_ids":  tokens
            }
        )

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:40<00:00, 24.93it/s]


## Summarize Docs
Split tokens in pages of 5 tokens

A single paper (document) is divided in pages (token_splits): each page is an array of tokens.

Documents_tokenized contains the list of papers that must be summarized, and it is an array of documents.

In [6]:
file = open("paper_splits.json", "w")
paper_splits = []

In [9]:
max_size = 512

counter = 0

for idx, row in tqdm(enumerate(paper_rows)):
    
    n_splits = math.ceil(len(row["input_ids"][0])/max_size)
    document_text_summary = ""
    token_count = 0
    for index in list(range(n_splits)):
        if(counter==100):
            break
        if(index != n_splits-1):
            token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device),
                                "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device)}

        else:
            token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:len(row['input_ids']['input_ids'][0])%max_size + index*max_size]).unsqueeze(0).to(device),
                            "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:len(row['input_ids']['input_ids'][0])%max_size + index*max_size]).unsqueeze(0).to(device)}

        if(token_splits["input_ids"].nelement() == 0):
          continue
        
       
   
    
        doc_token_summary = model.generate(input_ids=token_splits['input_ids'],
                            attention_mask=token_splits['attention_mask'],
                            min_length=16,
                            max_length=128)
        token_count += len(doc_token_summary[0])
        extracted_summary = tokenizer.decode(doc_token_summary[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        
        paper_splits.append({
            "link": row["link"],
            "body_split": tokenizer.decode(token_splits["input_ids"][0], skip_special_tokens=True, clean_up_tokenization_spaces=True),
            "summary": ".".join(extracted_summary.split(".")[0:-1])})
        
        if ("." in extracted_summary):
             document_text_summary += (".".join(extracted_summary.split(".")[0:-1])) + ".\n"
        else:
            document_text_summary += extracted_summary + ".\n"

        counter += 1
    
    # requests.post(f"{host}/updateArticle", json={
    #   "link": row["link"],
    #   "token_count": token_count,
    #   "summary": document_text_summary.replace("..", ".")
    # }) 


0it [00:00, ?it/s]

C:\Users\fisch\AppData\Local\Temp\ipykernel_14816\2832457497.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token_splits = { "input_ids": torch.tensor(row['input_ids']['input_ids'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device),
C:\Users\fisch\AppData\Local\Temp\ipykernel_14816\2832457497.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(row['input_ids']['attention_mask'][0][index*max_size:(index+1)*max_size]).unsqueeze(0).to(device)}
C:\Users\fisch\AppData\Local\Temp\ipykernel_14816\2832457497.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach()

In [11]:
import json
file.write(json.dumps(paper_splits))
file.close()